In [1]:
from email.mime.multipart import MIMEMultipart
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt
from email.mime.text import MIMEText
import base64
import requests

In [2]:
# Connect to Drive API
spreadsheet_id = '1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM'
url = 'https://docs.google.com/spreadsheets/d/1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM/'
emails = ['treasury@celsius.network',
'weekly.portfolio@celsius.network']

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = None
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Drive v3 API
SAMPLE_RANGE_NAME = 'Pivot_Data!A1:H'
request = service.spreadsheets().values().batchGet(
    spreadsheetId=spreadsheet_id,
    ranges=SAMPLE_RANGE_NAME,
    valueRenderOption='UNFORMATTED_VALUE')
response = request.execute()

In [3]:
df = pd.DataFrame(response['valueRanges'][0]['values']).dropna().reset_index(drop=True)
df.columns = df.loc[0]
df2 = df.loc[1:]
cond = (df2['USD Value']>10000000) & ((df2['APY'].isna())|(df2['APY']==0)) & (~df2['Category'].isin(['CEL Treasury','Posted Collateral','Mining','CEL Users','Investments','undeployed','underdeployed','Underdeployed', 'operational']))
error_table = df2[cond][['Coin','Category','Account','APY','USD Value']]
error_table['USD Value'] = error_table['USD Value'].map(lambda x: f"${int(x):,}")
error = error_table.shape[0]>0

In [4]:
# Connect to GMAIL API
client_id = "me"
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
creds = None
creds = Credentials.from_authorized_user_file('token_gmail_send.json', SCOPES)

service_gmail = build('gmail', 'v1', credentials=creds)

In [5]:
if error:
    html = """
    <html><body><p>Hi all,</p>
    Those following accounts have more than $10M deployed in and does not have any APY in the WaterFall"
    {table}

    <p> And the link to the google sheet:</p>
    <a href="{url}">Portfolio Waterfall</a>

    <p>Regards,</p>
    <p>CNS BOT</p>
    </body></html>
    """


    subject = '[Alert] Portfolio Waterfall ---- Missing APYs'
    to = 'gabriel.choukroun@celsius.network'
    sender = 'gabriel.choukroun@celsius.network'


    html_table = error_table.to_html(index=False,
                                  header=True,
                                  justify='center',
                                  col_space=70)

    html = html.format(table=html_table, url=url)

    message = MIMEText(html, 'html')
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    user_id = "me"
    raw_message = {
        'raw':base64.urlsafe_b64encode(message.as_string().encode("utf-8")).decode("utf-8")}
     #   'payload': {'mimeType': 'text/html'}}

    message = service_gmail.users().messages().send(userId=user_id, body=raw_message).execute()
    print('Message Id: %s' % message['id'])

Message Id: 18018ea4466b21ae
